In [1]:
import numpy as np
import cv2

import glob
import os
import json


In [17]:
def preprocessing(img, isEdge=True):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    __, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    if isEdge :
        img = cv2.Canny(img,150,200)        
    return img

def contourOK(cc):
    x, y, w, h = cv2.boundingRect(cc)
    area = w*h
    return area > 25

def get_contours(img):
    __ , contours, hierarchy  = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contours = [cc for cc in contours if contourOK(cc) ]
    return contours

def validate_box(contours, img, isDraw=True):
    boxs = [] 
    for cc in contours:
        box = list(cv2.boundingRect(cc))
        box.append(cv2.contourArea(cc))
        boxs.append(box)
    boxs = np.array(sorted(boxs, key=lambda x: x[4], reverse=False))
    boxs = np.delete(boxs, 4, 1)
    
    inner_box = []
    for box_a in boxs:
        for box_b in boxs:
            if (box_a != box_b).all() :
                if box_a[0] < box_b[0] and \
                box_a[1] < box_b[1] and \
                box_a[0]+box_a[2] > box_b[0]+box_b[2] and \
                box_a[1]+box_a[3] > box_b[1]+box_b[3] :
                    inner_box.append(box_b)
                    
    outer_box = [ box for box in boxs if not any(np.array_equal(box, inbox) for inbox in inner_box)]
    for box in outer_box:
        x, y, w, h = box.astype(np.int16)
        if isDraw:
            cv2.rectangle(img, (x,y), (x+w, y+h), (0, 0, 255), 1)
        
    return img, outer_box

def split_staff(binary, img):
    images = []
    padding = 5
    ih, iw, __ = img.shape
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    contours = get_contours(binary)
    for cc in contours:
        x, y, w, h = cv2.boundingRect(cc)
        y1 = y if y > padding else 0 
        y2 = y+h if y+h < ih-padding else y+h+padding
        if w > iw*0.70 and h > 70:
            images.append(img[y1:y2, 0:iw, :])
    return images

def draw_line(edges, img):
    ih, __, __ = img.shape
    #dist = min(200, ih*0.9)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=200, maxLineGap=50)
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 1)
    return img

def remove_line(edges, img):
    white = (255,255,255)
    ih, __, __ = img.shape
    #dist = min(200, ih*0.9)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=200, maxLineGap=50)
    h_lines = [[x1, y1, x2, y2] for x1, y1, x2, y2 in lines[:, 0, :]  if abs(y1 - y2) < 10] #remove vertical line
    h_lines= np.array(sorted(h_lines, key=lambda x: x[1]))
    
    last_idx = 0
    idx_list = np.where(np.diff(h_lines[:, 1]) < 5)[0]
    for i, idx in enumerate(idx_list):
        
        if (idx - last_idx) >= 3:
            x1, y1, x2, y2 = h_lines[:, 0][idx-1], h_lines[:, 1][idx-1], h_lines[:, 2][idx-1], h_lines[:, 3][idx-1] + 3
            cv2.rectangle(img, (x1, y1), (x2, y2), white, -1) 
            
        x1, y1, x2, y2 = h_lines[:, 0][idx], h_lines[:, 1][idx], h_lines[:, 2][idx+1], h_lines[:, 3][idx+1] 
        cv2.rectangle(img, (x1, y1), (x2, y2), white, -1)

        last_idx = idx
        
    if abs(h_lines[-1][1] - h_lines[-2][1]) > 10: #check if last line is not paired line
        x1, y1, x2, y2 = h_lines[-1]
        cv2.rectangle(img, (x1, y1), (x2, y2+3), white, -1)
        
    if abs(h_lines[0][1] - h_lines[1][1]) > 10: #check if first line is not paired line
        x1, y1, x2, y2 = h_lines[0]
        cv2.rectangle(img, (x1, y1), (x2, y2+3), white, -1)
    
    for i, hl in enumerate(h_lines):  #check if inner line is not paired line
        if (i > 0 and i < (len(h_lines) - 1)) and \
                    (abs(h_lines[i][1] - h_lines[i-1][1]) > 10 and \
                    abs(h_lines[i][1] - h_lines[i+1][1]) > 10):
            x1, y1, x2, y2 = h_lines[i]
            cv2.rectangle(img, (x1, y1), (x2, y2+3), white, -1)       

    return img

def morph_transform(img):
    img = cv2.bitwise_not(img)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    connected = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    connected = cv2.dilate(connected,kernel,iterations = 1)
    return connected

def segmented_crop(img, boxs):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    images = []
    for box in boxs:
        x, y, w, h = box.astype(np.int16)
        images.append(img[0:img.shape[0], x:x+w])
    return images

def sort_boxs(boxs):
    return np.array(sorted(boxs, key=lambda x: x[0], reverse=False))

def box_outline(img):
    h, w, c = img.shape
    white = (255,255,255)
    img = cv2.rectangle(img, (0,0), (3,h), white, -1)
    img = cv2.rectangle(img, (w-3,0), (w,h), white, -1)
    return img

In [18]:
def test_process(idx=0):
    
    filename = filenames[idx]
    num_ = int(filename.replace(".png", "")) - 1
#     print(labels[num_])
    
    img_original = cv2.imread(path + filename)
    
    binary_inv = cv2.bitwise_not(preprocessing(img_original.copy(), False))
    
    staff_imgs = split_staff(binary_inv.copy(), img_original.copy())
    cv2.imshow("binary-" + filename, binary_inv)
    
    for i, img in enumerate(staff_imgs):
        
        img = np.array(img).astype(np.uint8)
        
        #cv2.imshow("original_staff_%s_on_image_%s" %(i, filename), img.copy())
        
        img = box_outline(img.copy())
        
        edges = preprocessing(img.copy(), True)
        
        img_draw_line = draw_line(edges.copy(), img.copy())

        img_removed_line = remove_line(edges.copy(), img.copy())

        img_connected_obj = morph_transform(img_removed_line.copy())

        edges_clean = preprocessing(img_connected_obj)

        contours = get_contours(edges_clean.copy())

        img_contour = cv2.drawContours(img.copy(), contours, -1, (0, 255, 0), 1)

        img_box, boxs = validate_box(contours, img.copy(), isDraw=True)

        boxs = sort_boxs(boxs)

        segmented_image = segmented_crop(img.copy(), boxs)

#         for i, seg_im in enumerate(segmented_image) :
#             cv2.imshow("img_seg_" + str(i), seg_im)

        cv2.imshow("original-" + filename, img.copy())
        cv2.imshow("canny_edge-" + filename, edges.copy())
        cv2.imshow("img_line-" + filename, img_draw_line)
        cv2.imshow("bounding_box_staff_%s_on_image_%s" %(i, filename), img_box)
        cv2.imshow("morphological-" + filename, img_connected_obj)
        cv2.imshow("contour-" + filename, img_contour)
        cv2.waitKey(0)
        
    cv2.destroyAllWindows()

In [19]:
path = "data/lilypond/"
target_path = "Dataset/"

fp = open(path + 'labels.txt', 'r')
labels = fp.read().split('\n')

if labels[-1] == '':
    labels.pop()
fp.close()

labels = [ lbl.split(' ') for lbl in labels]
labels = [[item for item in label if item != ''] for label in labels]

list_file = os.listdir(path)
num_sample = len(list_file)

filenames = []
for filename in list_file:
    filenames.append(filename)

In [ ]:
for i in range(0,1):
    test_process(i)

In [193]:
for i, filename in enumerate(list_file[0:3]):
    img = cv2.imread(path + filename)
    edges = preprocessing(img)

    img_removed_line = remove_line(edges, img.copy())
    img_connected_obj = morph_transform(img_removed_line.copy())
    edges = preprocessing(img_connected_obj)

    contours = get_contours(edges)
    img_contour = cv2.drawContours(img.copy(), contours, -1, (0, 255, 0), 1)
    img_box, boxs = validate_box(contours, img.copy(), isDraw=True)
    boxs = sort_boxs(boxs)
    segmented_image = segmented_crop(img.copy(), boxs)

    for j, seg_im in enumerate(segmented_image) :
        cv2.imwrite(target_path + labels[i][j] + "_" + filename, seg_im)
    print("[INFO] processing file %s for %s to %d" % (filename, i, num_sample))

[INFO] processing file 1.png for 0 to 20003
[INFO] processing file 10.png for 1 to 20003
[INFO] processing file 100.png for 2 to 20003
